In [11]:
from langchain_community.chat_models import ChatOllama
from openai import OpenAI
import keyring

llm_llama = ChatOllama(model='llama3.1')
llm_openai = OpenAI(
    api_key=keyring.get_password('openai', 'key_for_windows')
)

def get_response_from_llama(prompt):
    response = llm_llama.invoke(prompt)
    return response.content

def get_response_from_openai(prompt):
    completion = llm_openai.chat.completions.create(
        messages=[
            {'role':'user', 'content':prompt}
        ],
        model='gpt-3.5-turbo',
        max_tokens=1024,
        temperature=0.0
    )
    return completion.choices[0].message.content



In [12]:
prompt = 'hi'
print(f"Llama: {get_response_from_llama(prompt)}")
print()
print(f"ChatGPT: {get_response_from_openai(prompt)}")

Llama: How's it going? Is there something I can help you with, or would you like to chat?

ChatGPT: Hello! How can I assist you today?


In [14]:
document = '''
Mark Zuckerberg laid out Meta's gameplan for "playing to win" against Alphabet and Microsoft in the high-stakes AI arms race. Meta's secret weapon: its walled garden of data.

"There are hundreds of billions of publicly shared images and tens of billions of public videos, which we estimate is greater than the common crawl data set,” Zuckerberg said on Meta’s earnings call on Thursday. It was a not-so-subtle jab at competitors Google, Microsoft and OpenAI, which are training their AI models on the public web data crawled by their search engines every day.
'''
prompt_base = f"""아래 문장에서 개체명 인식을 수행하세요.
####
{document}
"""
print(prompt_base)

아래 문장에서 개체명 인식을 수행하세요.
####

Mark Zuckerberg laid out Meta's gameplan for "playing to win" against Alphabet and Microsoft in the high-stakes AI arms race. Meta's secret weapon: its walled garden of data.

"There are hundreds of billions of publicly shared images and tens of billions of public videos, which we estimate is greater than the common crawl data set,” Zuckerberg said on Meta’s earnings call on Thursday. It was a not-so-subtle jab at competitors Google, Microsoft and OpenAI, which are training their AI models on the public web data crawled by their search engines every day.




In [15]:
answer_base_llama = get_response_from_llama(prompt_base)
print(answer_base_llama)

아래는 개체명 인식을 수행한 결과입니다.

* Mark Zuckerberg : 사람
* Meta : 회사(미국)
* Alphabet : 회사(미국)
* Microsoft : 회사(미국)


In [16]:
prompt_add_guide = f"""아래 문장에서 개체명 인식을 수행하세요.
아래 문장에서 사람 이름(PERSON), 기업 이름(ORG)에 해당하는 엔티티만을 추출하세요.
####
{document}
"""

answer_add_guide_llama = get_response_from_llama(prompt_add_guide)
print(answer_add_guide_llama)

다음은 주어진 문장에서 사람 이름(PERSON)과 기업 이름(ORG)을 추출한 결과입니다:

* 사람 이름(PERSON): 
  Mark Zuckerberg
* 기업 이름(ORG): 
  Meta 
  Alphabet 
  Microsoft 
  Google 
  OpenAI


In [18]:
prompt_output = f"""아래 문장에서 개체명 인식을 수행하세요.
아래 문장에서 사람 이름(PERSON), 기업 이름(ORG)에 해당하는 엔티티만을 추출하세요.
출력 결과는 분류결과(PERSON/ORG): alias 형식으로 출력하세요.
####
{document}
"""

answer_output_llama = get_response_from_llama(prompt_output)
print(answer_output_llama)

MARK ZUCKERBERG (PERSON): Mark Zuckerberg
META (ORG): Meta
ALPHABET (ORG): Alphabet
MICROSOFT (ORG): Microsoft 
GOOGLE (ORG): Google
OPENAI (ORG): OpenAI


In [19]:
prompt_improvement = f"""아래 문장에서 개체명 인식을 수행하세요.
아래 문장에서 사람 이름(PERSON), 기업 이름(ORG)에 해당하는 엔티티만을 추출하세요.
출력 결과는 alias: 분류결과(PERSON/ORG) 형식으로 한 줄에 한 결과씩 출력하세요.
####
{document}
"""

answer_improvement_llama = get_response_from_llama(prompt_improvement)
print(answer_improvement_llama)

Mark Zuckerberg: PERSON
Meta: ORG
Alphabet: ORG
Microsoft: ORG
OpenAI: ORG


In [20]:
prompt_json = f"""아래 문장에서 개체명 인식을 수행하세요.
아래 문장에서 사람 이름(PERSON), 기업 이름(ORG)에 해당하는 엔티티만을 추출하세요.
출력 결과는 json 포맷으로 alias: 분류결과(PERSON/ORG)를 출력하세요.
####
{document}
"""
answer_json_llama = get_response_from_llama(prompt_json)
print(answer_json_llama)

{"entities": [
	{
		"alias": "Mark Zuckerberg",
		"category": "PERSON"
	},
	{
		"alias": "Meta",
		"category": "ORG"
	},
	{
		"alias": "Alphabet",
		"category": "ORG"
	},
	{
		"alias": "Microsoft",
		"category": "ORG"
	},
	{
		"alias": "Google",
		"category": "ORG"
	},
	{
		"alias": "OpenAI",
		"category": "ORG"
	}
]}


In [22]:
prompt_json_2 = f"""아래 문장에서 개체명 인식을 수행하세요.
아래 문장에서 사람 이름(PERSON), 기업 이름(ORG)에 해당하는 엔티티만을 추출하세요.
출력 결과는 json 포맷으로 {{0:{{"text": alias, "entity":분류결과(PERSON/ORG)}}, 1:...}} 형식으로 출력하세요.
####
{document}
"""

answer_json_2_llama = get_response_from_llama(prompt_json_2)
print(answer_json_2_llama)

{0: {"text": "Mark Zuckerberg", "entity": "PERSON"}, 1: {"text": "Meta", "entity": "ORG"}, 
2: {"text": "Alphabet", "entity": "ORG"}, 
3: {"text": "Microsoft", "entity": "ORG"}, 
4: {"text": "Google", "entity": "ORG"}, 
5: {"text": "OpenAI", "entity": "ORG"}}


In [25]:
prompt_position = f"""아래 문장에서 개체명 인식을 수행하세요.
아래 문장에서 사람 이름(PERSON), 기업 이름(ORG)에 해당하는 엔티티만을 추출하세요.
엔터티의 문장 내 몇 번째 문자[start, end]인지 함께 출력하세요.
출력 결과는 json 포맷으로 {{0:{{"text": alias, "entity":분류결과(PERSON/ORG), "position": [0, 2]}}, 1:...}} 형식으로 출력하세요.
####
{document}
"""
answer_position_llama = get_response_from_llama(prompt_position)
print(answer_position_llama)

일단 개체명 인식을 수행한 결과입니다.

```json
[0:{"text": "Mark Zuckerberg", "entity":"PERSON", "position": [0, 16]},
1:{"text": "Meta", "entity":"ORG", "position":[29,34]},
2:{"text": "Alphabet", "entity":"ORG", "position":[56,65]},
3:{"text": "Microsoft", "entity":"ORG", "position":[68,79]},
4:{"text": "Mark Zuckerberg", "entity":"PERSON", "position":[86,102]}]
```
